# Tabulate results


In [1]:
import os
import sys
from typing import Tuple

import pandas as pd
from tabulate import tabulate
from tqdm import tqdm
sys.path.append('../src')
from read_log_file import read_log_file

In [2]:
LOG_HOME_DIR = os.path.join('../logs_v2')
assert os.path.isdir(LOG_HOME_DIR)

In [3]:
MODEL_NAMES = ['logistic_regression', 'transformer_encoder', 'bert-base-uncased', 'bert-base-multilingual-cased']

In [4]:
SETUPS = ['zero', 'few50', 'few100', 'few150', 'few200', 'full', 'trg']

In [5]:
def get_best_score_from_dict(di: dict) -> dict:
    """Get max value from a dict"""
    keys_with_max_val = []
    # find max value
    max_val = -float('inf')
    for k, v in di.items():
        if v > max_val:
            max_val = v
    # find all keys with max value
    for k, v in di.items():
        if v == max_val:
            keys_with_max_val.append(k)
    return {
        'k': keys_with_max_val,
        'v': max_val,
    }

In [6]:
def create_best_results_df(langs: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    results_dict = {}
    for model_name in MODEL_NAMES:
        results_dict[model_name] = {}
        log_dir = os.path.join(LOG_HOME_DIR, langs, model_name)
        log_filenames = os.listdir(log_dir)
        for fname in log_filenames:
            results_dict[model_name][fname] = read_log_file(
                log_file_path=os.path.join(log_dir, fname),
                plot=False,
                verbose=False,
            )['best_val_metrics']['f1']

    best_results_dict = {'Setup': SETUPS}
    best_hparams_dict = {'Setup': SETUPS}
    best_results_dict.update({model_name: [] for model_name in MODEL_NAMES})
    best_hparams_dict.update({model_name: [] for model_name in MODEL_NAMES})
    for model_name in MODEL_NAMES:
        for setup in SETUPS:
            best_score = get_best_score_from_dict(
                {k: v for k, v in results_dict[model_name].items() if k.startswith(f'{setup}_')}
            )
            best_results_dict[model_name].append(
                best_score['v']
            )
            best_hparams_dict[model_name].append(
                best_score['k']
            )


    best_results_df = pd.DataFrame(best_results_dict)
    best_hparams_df = pd.DataFrame(best_hparams_dict)
    return best_results_df, best_hparams_df

In [7]:
def highlight_best_score(df: pd.DataFrame) -> pd.DataFrame:
    """Highlight best score in each row"""
    return df.style.apply(lambda x: ['background: red' if isinstance(v, float) and v == max(x.iloc[1:]) else '' for v in x], axis=1)

In [8]:
def tabulate_markdown(df: pd.DataFrame) -> str:
    """Tabulate in markdown format and bold best scores in each row"""
    df = df.round(4)
    for model_name in MODEL_NAMES:
        df[model_name] = df[model_name].astype(str)
    for idx in range(len(df)):
        max_val = max(float(df.iloc[idx][model_name]) for model_name in MODEL_NAMES)
        for model_name in MODEL_NAMES:
            cell_val = float(df.iloc[idx][model_name])
            if cell_val == max_val:
                df.at[idx, model_name] = f'**{cell_val}**'
            else:
                df.at[idx, model_name] = f'{cell_val}'

    return tabulate(df, headers='keys', showindex=False, tablefmt='github')


In [9]:
def tabulate_latex(df: pd.DataFrame) -> str:
    """Tabulate in markdown format and bold best scores in each row"""
    df = df.round(4)
    for model_name in MODEL_NAMES:
        df[model_name] = df[model_name].astype(str)
    for idx in range(len(df)):
        max_val = max(float(df.iloc[idx][model_name]) for model_name in MODEL_NAMES)
        for model_name in MODEL_NAMES:
            cell_val = float(df.iloc[idx][model_name])
            if cell_val == max_val:
                df.at[idx, model_name] = f'{cell_val}'
            else:
                df.at[idx, model_name] = f'{cell_val}'

    return tabulate(df, headers='keys', showindex=False, tablefmt='latex')

In [10]:
best_results_dfs_dict = {}
best_hparams_dfs_dict = {}
for langs in tqdm(['enbg', 'enar', 'bgen', 'bgar', 'aren', 'arbg']):
    best_results_dfs_dict[langs], best_hparams_dfs_dict[langs] = create_best_results_df(langs)

100%|██████████| 6/6 [00:21<00:00,  3.61s/it]


## en-bg

In [11]:
highlight_best_score(best_results_dfs_dict['enbg'])

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,0.369656,0.803699,0.802720,0.810053
1,few50,0.775659,0.799576,0.810351,0.818590
2,few100,0.781497,0.810259,0.816115,0.822725
3,few150,0.795635,0.819153,0.818532,0.820666
4,few200,0.807414,0.819964,0.815694,0.824915
5,full,0.812499,0.814600,0.822458,0.833575
6,trg,0.813775,0.809617,0.821763,0.843117


In [12]:
print(tabulate_markdown(best_results_dfs_dict['enbg']))

| Setup   |   logistic_regression |   transformer_encoder |   bert-base-uncased | bert-base-multilingual-cased   |
|---------|-----------------------|-----------------------|---------------------|--------------------------------|
| zero    |                0.3697 |                0.8037 |              0.8027 | **0.8101**                     |
| few50   |                0.7757 |                0.7996 |              0.8104 | **0.8186**                     |
| few100  |                0.7815 |                0.8103 |              0.8161 | **0.8227**                     |
| few150  |                0.7956 |                0.8192 |              0.8185 | **0.8207**                     |
| few200  |                0.8074 |                0.82   |              0.8157 | **0.8249**                     |
| full    |                0.8125 |                0.8146 |              0.8225 | **0.8336**                     |
| trg     |                0.8138 |                0.8096 |              0.8218 

In [13]:
print(tabulate_latex(best_results_dfs_dict['enbg']))

\begin{tabular}{lrrrr}
\hline
 Setup   &   logistic\_regression &   transformer\_encoder &   bert-base-uncased &   bert-base-multilingual-cased \\
\hline
 zero    &                0.3697 &                0.8037 &              0.8027 &                         0.8101 \\
 few50   &                0.7757 &                0.7996 &              0.8104 &                         0.8186 \\
 few100  &                0.7815 &                0.8103 &              0.8161 &                         0.8227 \\
 few150  &                0.7956 &                0.8192 &              0.8185 &                         0.8207 \\
 few200  &                0.8074 &                0.82   &              0.8157 &                         0.8249 \\
 full    &                0.8125 &                0.8146 &              0.8225 &                         0.8336 \\
 trg     &                0.8138 &                0.8096 &              0.8218 &                         0.8431 \\
\hline
\end{tabular}


In [14]:
best_hparams_dfs_dict['enbg']

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,"[zero_hidden256_vocab16000.txt, zero_hidden256...","[zero_hidden512_vocab16000.txt, zero_hidden512...","[zero_fc128_lr0.0005_trainable.txt, zero_fc128...",[zero_fc512_lr0.005_frozen.txt]
1,few50,"[few50_hidden512_vocab16000.txt, few50_hidden5...","[few50_hidden512_vocab16000.txt, few50_hidden5...",[few50_fc128_lr0.0005_frozen.txt],[few50_fc256_lr0.005_frozen.txt]
2,few100,"[few100_hidden512_vocab16000.txt, few100_hidde...","[few100_hidden256_vocab16000.txt, few100_hidde...",[few100_fc128_lr0.005_frozen.txt],[few100_fc512_lr0.005_frozen.txt]
3,few150,"[few150_hidden256_vocab16000.txt, few150_hidde...","[few150_hidden256_vocab16000.txt, few150_hidde...",[few150_fc512_lr0.0005_frozen.txt],[few150_fc128_lr0.005_frozen.txt]
4,few200,"[few200_hidden128_vocab16000.txt, few200_hidde...","[few200_hidden128_vocab16000.txt, few200_hidde...",[few200_fc256_lr0.0005_frozen.txt],[few200_fc512_lr0.005_frozen.txt]
5,full,"[full_hidden256_vocab16000.txt, full_hidden256...","[full_hidden512_vocab16000.txt, full_hidden512...",[full_fc512_lr0.0005_frozen.txt],[full_fc256_lr0.0005_frozen.txt]
6,trg,"[trg_hidden256_vocab16000.txt, trg_hidden256_v...","[trg_hidden256_vocab16000.txt, trg_hidden256_v...",[trg_fc128_lr0.0005_frozen.txt],[trg_fc512_lr0.0005_frozen.txt]


## en-ar

In [15]:
highlight_best_score(best_results_dfs_dict['enar'])

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,0.422418,0.584852,0.598659,0.672059
1,few50,0.727199,0.689753,0.675045,0.774892
2,few100,0.743307,0.685501,0.691503,0.824014
3,few150,0.718456,0.688997,0.697930,0.791109
4,few200,0.747128,0.739566,0.711928,0.786594
5,full,0.747128,0.739566,0.711928,0.786594
6,trg,0.648866,0.684390,0.734822,0.738182


In [16]:
print(tabulate_markdown(best_results_dfs_dict['enar']))

| Setup   |   logistic_regression |   transformer_encoder |   bert-base-uncased | bert-base-multilingual-cased   |
|---------|-----------------------|-----------------------|---------------------|--------------------------------|
| zero    |                0.4224 |                0.5849 |              0.5987 | **0.6721**                     |
| few50   |                0.7272 |                0.6898 |              0.675  | **0.7749**                     |
| few100  |                0.7433 |                0.6855 |              0.6915 | **0.824**                      |
| few150  |                0.7185 |                0.689  |              0.6979 | **0.7911**                     |
| few200  |                0.7471 |                0.7396 |              0.7119 | **0.7866**                     |
| full    |                0.7471 |                0.7396 |              0.7119 | **0.7866**                     |
| trg     |                0.6489 |                0.6844 |              0.7348 

In [17]:
print(tabulate_latex(best_results_dfs_dict['enar']))

\begin{tabular}{lrrrr}
\hline
 Setup   &   logistic\_regression &   transformer\_encoder &   bert-base-uncased &   bert-base-multilingual-cased \\
\hline
 zero    &                0.4224 &                0.5849 &              0.5987 &                         0.6721 \\
 few50   &                0.7272 &                0.6898 &              0.675  &                         0.7749 \\
 few100  &                0.7433 &                0.6855 &              0.6915 &                         0.824  \\
 few150  &                0.7185 &                0.689  &              0.6979 &                         0.7911 \\
 few200  &                0.7471 &                0.7396 &              0.7119 &                         0.7866 \\
 full    &                0.7471 &                0.7396 &              0.7119 &                         0.7866 \\
 trg     &                0.6489 &                0.6844 &              0.7348 &                         0.7382 \\
\hline
\end{tabular}


In [18]:
best_hparams_dfs_dict['enar']

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,"[zero_hidden128_vocab16000.txt, zero_hidden128...","[zero_hidden512_vocab16000.txt, zero_hidden512...",[zero_fc128_lr0.0005_frozen.txt],[zero_fc512_lr0.005_frozen.txt]
1,few50,"[few50_hidden512_vocab16000.txt, few50_hidden5...","[few50_hidden128_vocab16000.txt, few50_hidden1...",[few50_fc128_lr0.0005_frozen.txt],[few50_fc128_lr0.0005_frozen.txt]
2,few100,"[few100_hidden256_vocab16000.txt, few100_hidde...","[few100_hidden128_vocab16000.txt, few100_hidde...",[few100_fc256_lr0.0005_frozen.txt],[few100_fc512_lr0.005_frozen.txt]
3,few150,"[few150_hidden512_vocab16000.txt, few150_hidde...","[few150_hidden512_vocab16000.txt, few150_hidde...",[few150_fc512_lr0.005_frozen.txt],[few150_fc512_lr0.0005_frozen.txt]
4,few200,"[few200_hidden512_vocab16000.txt, few200_hidde...","[few200_hidden512_vocab16000.txt, few200_hidde...",[few200_fc256_lr0.0005_frozen.txt],[few200_fc512_lr0.005_frozen.txt]
5,full,"[full_hidden512_vocab16000.txt, full_hidden512...","[full_hidden512_vocab16000.txt, full_hidden512...",[full_fc256_lr0.0005_frozen.txt],[full_fc512_lr0.005_frozen.txt]
6,trg,"[trg_hidden128_vocab16000.txt, trg_hidden128_v...","[trg_hidden256_vocab16000.txt, trg_hidden256_v...",[trg_fc512_lr0.0005_frozen.txt],[trg_fc512_lr0.005_frozen.txt]


## bg-en

In [19]:
highlight_best_score(best_results_dfs_dict['bgen'])

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,0.522791,0.569252,0.487918,0.684943
1,few50,0.600851,0.577882,0.642544,0.671831
2,few100,0.607478,0.618867,0.620778,0.658960
3,few150,0.635134,0.609627,0.631913,0.655309
4,few200,0.674276,0.635436,0.712940,0.696439
5,full,0.713038,0.670283,0.728505,0.699864
6,trg,0.698497,0.686484,0.744600,0.722145


In [20]:
print(tabulate_markdown(best_results_dfs_dict['bgen']))

| Setup   |   logistic_regression |   transformer_encoder | bert-base-uncased   | bert-base-multilingual-cased   |
|---------|-----------------------|-----------------------|---------------------|--------------------------------|
| zero    |                0.5228 |                0.5693 | 0.4879              | **0.6849**                     |
| few50   |                0.6009 |                0.5779 | 0.6425              | **0.6718**                     |
| few100  |                0.6075 |                0.6189 | 0.6208              | **0.659**                      |
| few150  |                0.6351 |                0.6096 | 0.6319              | **0.6553**                     |
| few200  |                0.6743 |                0.6354 | **0.7129**          | 0.6964                         |
| full    |                0.713  |                0.6703 | **0.7285**          | 0.6999                         |
| trg     |                0.6985 |                0.6865 | **0.7446**          

In [21]:
print(tabulate_latex(best_results_dfs_dict['bgen']))

\begin{tabular}{lrrrr}
\hline
 Setup   &   logistic\_regression &   transformer\_encoder &   bert-base-uncased &   bert-base-multilingual-cased \\
\hline
 zero    &                0.5228 &                0.5693 &              0.4879 &                         0.6849 \\
 few50   &                0.6009 &                0.5779 &              0.6425 &                         0.6718 \\
 few100  &                0.6075 &                0.6189 &              0.6208 &                         0.659  \\
 few150  &                0.6351 &                0.6096 &              0.6319 &                         0.6553 \\
 few200  &                0.6743 &                0.6354 &              0.7129 &                         0.6964 \\
 full    &                0.713  &                0.6703 &              0.7285 &                         0.6999 \\
 trg     &                0.6985 &                0.6865 &              0.7446 &                         0.7221 \\
\hline
\end{tabular}


In [22]:
best_hparams_dfs_dict['bgen']

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,"[zero_hidden128_vocab16000.txt, zero_hidden128...","[zero_hidden256_vocab16000.txt, zero_hidden256...",[zero_fc256_lr0.05_frozen.txt],[zero_fc512_lr0.0005_frozen.txt]
1,few50,"[few50_hidden128_vocab16000.txt, few50_hidden1...","[few50_hidden128_vocab16000.txt, few50_hidden1...",[few50_fc512_lr0.0005_frozen.txt],[few50_fc128_lr0.0005_frozen.txt]
2,few100,"[few100_hidden128_vocab16000.txt, few100_hidde...","[few100_hidden128_vocab16000.txt, few100_hidde...",[few100_fc256_lr0.0005_frozen.txt],[few100_fc256_lr0.0005_frozen.txt]
3,few150,"[few150_hidden512_vocab16000.txt, few150_hidde...","[few150_hidden128_vocab16000.txt, few150_hidde...",[few150_fc512_lr0.0005_frozen.txt],[few150_fc512_lr0.0005_frozen.txt]
4,few200,"[few200_hidden128_vocab16000.txt, few200_hidde...","[few200_hidden256_vocab16000.txt, few200_hidde...",[few200_fc512_lr0.0005_frozen.txt],[few200_fc256_lr0.0005_frozen.txt]
5,full,"[full_hidden512_vocab16000.txt, full_hidden512...","[full_hidden128_vocab16000.txt, full_hidden128...",[full_fc512_lr0.005_frozen.txt],[full_fc256_lr0.005_frozen.txt]
6,trg,"[trg_hidden512_vocab16000.txt, trg_hidden512_v...","[trg_hidden128_vocab16000.txt, trg_hidden128_v...",[trg_fc256_lr0.005_frozen.txt],[trg_fc256_lr0.005_frozen.txt]


## bg-ar

In [23]:
highlight_best_score(best_results_dfs_dict['bgar'])

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,0.547247,0.614746,0.558164,0.638295
1,few50,0.676276,0.647330,0.657497,0.760313
2,few100,0.697810,0.733819,0.662149,0.753031
3,few150,0.725884,0.687609,0.651926,0.775172
4,few200,0.707694,0.715821,0.679205,0.764396
5,full,0.707694,0.715821,0.679205,0.764396
6,trg,0.648866,0.684390,0.734822,0.738182


In [24]:
print(tabulate_markdown(best_results_dfs_dict['bgar']))

| Setup   |   logistic_regression |   transformer_encoder |   bert-base-uncased | bert-base-multilingual-cased   |
|---------|-----------------------|-----------------------|---------------------|--------------------------------|
| zero    |                0.5472 |                0.6147 |              0.5582 | **0.6383**                     |
| few50   |                0.6763 |                0.6473 |              0.6575 | **0.7603**                     |
| few100  |                0.6978 |                0.7338 |              0.6621 | **0.753**                      |
| few150  |                0.7259 |                0.6876 |              0.6519 | **0.7752**                     |
| few200  |                0.7077 |                0.7158 |              0.6792 | **0.7644**                     |
| full    |                0.7077 |                0.7158 |              0.6792 | **0.7644**                     |
| trg     |                0.6489 |                0.6844 |              0.7348 

In [25]:
print(tabulate_latex(best_results_dfs_dict['bgar']))

\begin{tabular}{lrrrr}
\hline
 Setup   &   logistic\_regression &   transformer\_encoder &   bert-base-uncased &   bert-base-multilingual-cased \\
\hline
 zero    &                0.5472 &                0.6147 &              0.5582 &                         0.6383 \\
 few50   &                0.6763 &                0.6473 &              0.6575 &                         0.7603 \\
 few100  &                0.6978 &                0.7338 &              0.6621 &                         0.753  \\
 few150  &                0.7259 &                0.6876 &              0.6519 &                         0.7752 \\
 few200  &                0.7077 &                0.7158 &              0.6792 &                         0.7644 \\
 full    &                0.7077 &                0.7158 &              0.6792 &                         0.7644 \\
 trg     &                0.6489 &                0.6844 &              0.7348 &                         0.7382 \\
\hline
\end{tabular}


In [26]:
best_hparams_dfs_dict['bgar']

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,"[zero_hidden128_vocab16000.txt, zero_hidden128...","[zero_hidden256_vocab16000.txt, zero_hidden256...","[zero_fc128_lr0.0005_trainable.txt, zero_fc128...",[zero_fc128_lr0.005_frozen.txt]
1,few50,"[few50_hidden512_vocab16000.txt, few50_hidden5...","[few50_hidden256_vocab16000.txt, few50_hidden2...",[few50_fc256_lr0.0005_frozen.txt],[few50_fc512_lr0.0005_frozen.txt]
2,few100,"[few100_hidden512_vocab16000.txt, few100_hidde...","[few100_hidden256_vocab16000.txt, few100_hidde...",[few100_fc512_lr0.0005_frozen.txt],[few100_fc256_lr0.0005_frozen.txt]
3,few150,"[few150_hidden256_vocab16000.txt, few150_hidde...","[few150_hidden512_vocab16000.txt, few150_hidde...",[few150_fc128_lr0.0005_frozen.txt],[few150_fc512_lr0.0005_frozen.txt]
4,few200,"[few200_hidden256_vocab16000.txt, few200_hidde...","[few200_hidden512_vocab16000.txt, few200_hidde...",[few200_fc128_lr0.0005_frozen.txt],[few200_fc256_lr0.0005_frozen.txt]
5,full,"[full_hidden256_vocab16000.txt, full_hidden256...","[full_hidden512_vocab16000.txt, full_hidden512...",[full_fc128_lr0.0005_frozen.txt],[full_fc256_lr0.0005_frozen.txt]
6,trg,"[trg_hidden128_vocab16000.txt, trg_hidden128_v...","[trg_hidden256_vocab16000.txt, trg_hidden256_v...",[trg_fc512_lr0.0005_frozen.txt],[trg_fc512_lr0.005_frozen.txt]


## ar-en

In [27]:
highlight_best_score(best_results_dfs_dict['aren'])

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,0.435813,0.516790,0.593767,0.683008
1,few50,0.536712,0.553041,0.630206,0.658884
2,few100,0.622074,0.608997,0.639147,0.662509
3,few150,0.626780,0.610483,0.728866,0.664647
4,few200,0.661438,0.686012,0.721739,0.679520
5,full,0.715435,0.680290,0.745004,0.711777
6,trg,0.698497,0.686484,0.744600,0.722145


In [28]:
print(tabulate_markdown(best_results_dfs_dict['aren']))

| Setup   |   logistic_regression |   transformer_encoder | bert-base-uncased   | bert-base-multilingual-cased   |
|---------|-----------------------|-----------------------|---------------------|--------------------------------|
| zero    |                0.4358 |                0.5168 | 0.5938              | **0.683**                      |
| few50   |                0.5367 |                0.553  | 0.6302              | **0.6589**                     |
| few100  |                0.6221 |                0.609  | 0.6391              | **0.6625**                     |
| few150  |                0.6268 |                0.6105 | **0.7289**          | 0.6646                         |
| few200  |                0.6614 |                0.686  | **0.7217**          | 0.6795                         |
| full    |                0.7154 |                0.6803 | **0.745**           | 0.7118                         |
| trg     |                0.6985 |                0.6865 | **0.7446**          

In [29]:
print(tabulate_latex(best_results_dfs_dict['aren']))

\begin{tabular}{lrrrr}
\hline
 Setup   &   logistic\_regression &   transformer\_encoder &   bert-base-uncased &   bert-base-multilingual-cased \\
\hline
 zero    &                0.4358 &                0.5168 &              0.5938 &                         0.683  \\
 few50   &                0.5367 &                0.553  &              0.6302 &                         0.6589 \\
 few100  &                0.6221 &                0.609  &              0.6391 &                         0.6625 \\
 few150  &                0.6268 &                0.6105 &              0.7289 &                         0.6646 \\
 few200  &                0.6614 &                0.686  &              0.7217 &                         0.6795 \\
 full    &                0.7154 &                0.6803 &              0.745  &                         0.7118 \\
 trg     &                0.6985 &                0.6865 &              0.7446 &                         0.7221 \\
\hline
\end{tabular}


In [30]:
best_hparams_dfs_dict['aren']

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,"[zero_hidden128_vocab16000.txt, zero_hidden128...","[zero_hidden256_vocab16000.txt, zero_hidden256...",[zero_fc512_lr0.0005_frozen.txt],[zero_fc512_lr0.005_frozen.txt]
1,few50,"[few50_hidden256_vocab16000.txt, few50_hidden2...","[few50_hidden128_vocab16000.txt, few50_hidden1...",[few50_fc512_lr0.005_frozen.txt],[few50_fc256_lr0.005_frozen.txt]
2,few100,"[few100_hidden512_vocab16000.txt, few100_hidde...","[few100_hidden512_vocab16000.txt, few100_hidde...",[few100_fc128_lr0.005_frozen.txt],[few100_fc512_lr0.005_frozen.txt]
3,few150,"[few150_hidden512_vocab16000.txt, few150_hidde...","[few150_hidden128_vocab16000.txt, few150_hidde...",[few150_fc512_lr0.005_frozen.txt],[few150_fc512_lr0.0005_frozen.txt]
4,few200,"[few200_hidden256_vocab16000.txt, few200_hidde...","[few200_hidden128_vocab16000.txt, few200_hidde...",[few200_fc512_lr0.0005_frozen.txt],[few200_fc256_lr0.005_frozen.txt]
5,full,"[full_hidden256_vocab16000.txt, full_hidden256...","[full_hidden512_vocab16000.txt, full_hidden512...",[full_fc512_lr0.0005_frozen.txt],[full_fc512_lr0.005_frozen.txt]
6,trg,"[trg_hidden512_vocab16000.txt, trg_hidden512_v...","[trg_hidden128_vocab16000.txt, trg_hidden128_v...",[trg_fc256_lr0.005_frozen.txt],[trg_fc256_lr0.005_frozen.txt]


## ar-bg

In [31]:
highlight_best_score(best_results_dfs_dict['arbg'])

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,0.558355,0.805075,0.802720,0.808254
1,few50,0.794193,0.803829,0.810778,0.815238
2,few100,0.799189,0.808224,0.818481,0.821288
3,few150,0.800414,0.816061,0.819586,0.820654
4,few200,0.800421,0.811238,0.822103,0.819945
5,full,0.820994,0.812141,0.821548,0.836257
6,trg,0.813775,0.809617,0.821763,0.843117


In [32]:
print(tabulate_markdown(best_results_dfs_dict['arbg']))

| Setup   |   logistic_regression |   transformer_encoder | bert-base-uncased   | bert-base-multilingual-cased   |
|---------|-----------------------|-----------------------|---------------------|--------------------------------|
| zero    |                0.5584 |                0.8051 | 0.8027              | **0.8083**                     |
| few50   |                0.7942 |                0.8038 | 0.8108              | **0.8152**                     |
| few100  |                0.7992 |                0.8082 | 0.8185              | **0.8213**                     |
| few150  |                0.8004 |                0.8161 | 0.8196              | **0.8207**                     |
| few200  |                0.8004 |                0.8112 | **0.8221**          | 0.8199                         |
| full    |                0.821  |                0.8121 | 0.8215              | **0.8363**                     |
| trg     |                0.8138 |                0.8096 | 0.8218              

In [33]:
print(tabulate_latex(best_results_dfs_dict['arbg']))



\begin{tabular}{lrrrr}
\hline
 Setup   &   logistic\_regression &   transformer\_encoder &   bert-base-uncased &   bert-base-multilingual-cased \\
\hline
 zero    &                0.5584 &                0.8051 &              0.8027 &                         0.8083 \\
 few50   &                0.7942 &                0.8038 &              0.8108 &                         0.8152 \\
 few100  &                0.7992 &                0.8082 &              0.8185 &                         0.8213 \\
 few150  &                0.8004 &                0.8161 &              0.8196 &                         0.8207 \\
 few200  &                0.8004 &                0.8112 &              0.8221 &                         0.8199 \\
 full    &                0.821  &                0.8121 &              0.8215 &                         0.8363 \\
 trg     &                0.8138 &                0.8096 &              0.8218 &                         0.8431 \\
\hline
\end{tabular}


In [34]:
best_hparams_dfs_dict['arbg']

,Setup,logistic_regression,transformer_encoder,bert-base-uncased,bert-base-multilingual-cased
0,zero,"[zero_hidden128_vocab16000.txt, zero_hidden128...","[zero_hidden512_vocab16000.txt, zero_hidden512...","[zero_fc128_lr0.0005_trainable.txt, zero_fc128...",[zero_fc256_lr0.005_frozen.txt]
1,few50,"[few50_hidden256_vocab16000.txt, few50_hidden2...","[few50_hidden256_vocab16000.txt, few50_hidden2...",[few50_fc128_lr0.0005_frozen.txt],[few50_fc512_lr0.0005_frozen.txt]
2,few100,"[few100_hidden512_vocab16000.txt, few100_hidde...","[few100_hidden256_vocab16000.txt, few100_hidde...",[few100_fc128_lr0.0005_frozen.txt],[few100_fc128_lr0.005_frozen.txt]
3,few150,"[few150_hidden512_vocab16000.txt, few150_hidde...","[few150_hidden512_vocab16000.txt, few150_hidde...",[few150_fc128_lr0.0005_frozen.txt],[few150_fc256_lr0.0005_frozen.txt]
4,few200,"[few200_hidden512_vocab16000.txt, few200_hidde...","[few200_hidden512_vocab16000.txt, few200_hidde...",[few200_fc256_lr0.005_frozen.txt],[few200_fc128_lr0.0005_frozen.txt]
5,full,"[full_hidden256_vocab16000.txt, full_hidden256...","[full_hidden128_vocab16000.txt, full_hidden128...",[full_fc256_lr0.0005_frozen.txt],[full_fc512_lr0.0005_frozen.txt]
6,trg,"[trg_hidden256_vocab16000.txt, trg_hidden256_v...","[trg_hidden256_vocab16000.txt, trg_hidden256_v...",[trg_fc128_lr0.0005_frozen.txt],[trg_fc512_lr0.0005_frozen.txt]
